In [1]:
!pip install pylabwons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 62.9 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=48af528fe5619469310bf109130cc1a79eb1a31ead22deaebc21b519ad21068d
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have nump

In [1]:
import pylabwons as lw
import os
IS_COLAB = any([key.startswith('COLAB') for key in os.environ])
if IS_COLAB:
    from google.colab import files
else:
    from scripts.env import PATH

# WICS CLASSIFICATION

In [ ]:
wics = lw.WiseICS()
wics.fetch()
print(wics.logger)
if IS_COLAB:
    wics.to_parquet('wics.parquet', engine='pyarrow')
    files.download('wics.parquet')
else:
    wics.to_parquet(PATH.PARQ.WICS, engine='pyarrow')

# AFTER MARKET

In [ ]:
market = lw.AfterMarket()
market.fetch()
print(market.logger)
if IS_COLAB:
    market.to_parquet('aftermarket.parquet', engine='pyarrow')
    files.download('aftermarket.parquet')
else:
    market.to_parquet(PATH.PARQ.AFTERMARKET, engine='pyarrow')

# FUNDAMENTAL

In [5]:
base = wics.drop(columns=['date']).join(market.drop(columns=['name']))
target = base[base['marketCap'] >= base['marketCap'].median()]

In [ ]:
fundamental = lw.Fundamentals()
fundamental.fetch(*target.index)
print(fundamental.logger)
for col in fundamental.columns:
    if fundamental[col].dtype == 'object':
        fundamental[col] = fundamental[col].astype(str)
if IS_COLAB:
    fundamental.to_parquet('fundamentals.parquet', engine='pyarrow')
    files.download('fundamentals.parquet')
else:
    fundamental.to_parquet(PATH.PARQ.FUNDAMENTALS, engine='pyarrow')